In [77]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))
stance = pd.read_csv("//Users//kelvin//Downloads//data.csv", header=0,)
stance.head(10)

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,discuss
1,Hundreds of Palestinians flee floods in Gaza a...,158,discuss
2,"Christian Bale passes on role of Steve Jobs, a...",137,disagree
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,disagree
4,Spider burrowed through tourist's stomach and ...,1923,agree
5,'Nasa Confirms Earth Will Experience 6 Days of...,154,discuss
6,Accused Boston Marathon Bomber Severely Injure...,962,unrelated
7,Identity of ISIS terrorist known as 'Jihadi Jo...,2033,agree
8,Banksy 'Arrested & Real Identity Revealed' Is ...,1739,agree
9,British Aid Worker Confirmed Murdered By ISIS,882,agree


In [79]:
stance.drop('Body ID' , axis=1, inplace=True)

In [84]:
stance.head(1)

,Headline,Stance,label_num
0,Police find mass graves with at least '15 bodi...,discuss,2


In [81]:
# examine the class distribution
stance.Stance.value_counts().sort_index()


agree        12451
disagree     12513
discuss      12527
unrelated    12481
Name: Stance, dtype: int64

In [146]:

# how to define X and y (from the SMS data) for use with COUNTVECTORIZER
X = stance.Headline
y = stance.label_num
print(X.shape)
print(y.shape)

(49972,)
(49972,)


In [148]:
stance.head(3)

,Headline,Stance,label_num
0,Police find mass graves with at least '15 bodi...,discuss,2
1,Hundreds of Palestinians flee floods in Gaza a...,discuss,2
2,"Christian Bale passes on role of Steve Jobs, a...",disagree,1


In [145]:
# convert label to a numerical variable
stance['label_num'] = stance.Stance.map({'agree':0, 'disagree':1, 'discuss':2, 'unrelated':3})

In [159]:
news_bias_con = stance[(stance.label_num==0) | (stance.label_num==1)]

In [163]:
news_bias_con.shape

(24964, 3)

In [161]:
X = news_bias_con.Headline
y = news_bias_con.label_num

In [162]:
# split X and y into training and testing sets
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(18723,)
(6241,)
(18723,)
(6241,)


In [150]:
# instantiate the vectorizer
vect = CountVectorizer()

In [151]:
# learn training data vocabulary, then use it to create a document-term matrix
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)

In [152]:
# equivalently: combine fit and transform into a single step
X_train_dtm = vect.fit_transform(X_train)

In [153]:
# examine the document-term matrix
X_train_dtm

<37479x3354 sparse matrix of type '<class 'numpy.int64'>'
	with 407732 stored elements in Compressed Sparse Row format>

In [154]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)
X_test_dtm

<12493x3354 sparse matrix of type '<class 'numpy.int64'>'
	with 135517 stored elements in Compressed Sparse Row format>

In [155]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [156]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
%time nb.fit(X_train_dtm, y_train)

CPU times: user 13.4 ms, sys: 3.91 ms, total: 17.3 ms
Wall time: 19.8 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [157]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [138]:
from sklearn.ensemble import RandomForestClassifier
#forest = RandomForestClassifier(n_estimators = 100)
forest = RandomForestClassifier(max_depth = 10, min_samples_split=2, n_estimators = 100, random_state = 1)
my_forest = forest.fit(X_train_dtm, y_train)

In [141]:
print(my_forest.score(X_train_dtm, y_train))

0.574801046841


In [158]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.2497398543184183

In [111]:
# print the confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)

array([[1421, 1642],
       [1495, 1683]])

In [142]:
# print the classification report
print(metrics.classification_report(y_test, y_pred_class))

             precision    recall  f1-score   support

          0       0.49      0.50      0.50      3063
          1       0.51      0.50      0.51      3178

avg / total       0.50      0.50      0.50      6241



In [112]:
# print message text for the false positives (ham incorrectly classified as spam)
X_test[y_test < y_pred_class]

36982    Lego letter from the 1970s still offers a powe...
16109    'Sex attacker' has penis cut off with a cleave...
39426    In Kim Jong Un's absence, his sister may rule....
11378    Apple Watch to Be Shower-Proof, Have 100,000 A...
17517         Cheese addiction breaks Kim Jong-un's ankles
15078    Officials Refute Iraqi Media Reports That ISIS...
23091    Kim Jong-un: obese leader 'fractured both ankles'
8704      U.S. arms airdrop fell into ISIS hands: Pentagon
39231    North Korean leader Kim Jong-un to open restau...
39992    Shots Heard In Alleged Brown Shooting Recordin...
11506    Officials Refute Iraqi Media Reports That ISIS...
42965    New policy could reduce marijuana possession a...
37679    Tropical spider burrows under man's skin throu...
45004    Christian Bale in Talks to Play Steve Jobs (Ex...
8643     CNN Broadcasts Purported Audio Containing Guns...
26260    6 hidden mass graves may hold missing Mexican ...
18703    ISIS pays tribute to its 'Cub of Baghdadi' chi.

In [113]:
# calculate predicted probabilities for X_test_dtm (poorly calibrated)
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

array([ 0.3219063 ,  0.43586059,  0.17301991, ...,  0.00074338,
        0.73071962,  0.71983502])

In [114]:
# calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)

0.49810811638207253

In [115]:
# import and instantiate a logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [116]:
# train the model using X_train_dtm
%time logreg.fit(X_train_dtm, y_train)

CPU times: user 341 ms, sys: 6.85 ms, total: 348 ms
Wall time: 349 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [117]:
# make class predictions for X_test_dtm
y_pred_class = logreg.predict(X_test_dtm)

In [118]:
# calculate predicted probabilities for X_test_dtm (well calibrated)
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

array([ 0.50056892,  0.46088328,  0.46005781, ...,  0.25724169,
        0.65132878,  0.52777993])

In [119]:
# calculate accuracy
metrics.accuracy_score(y_test, y_pred_class)

0.50216311488543508

In [120]:
# calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)

0.49901758888801906

In [121]:
# store the vocabulary of X_train
X_train_tokens = vect.get_feature_names()
len(X_train_tokens)

3329

In [122]:
# examine the first 50 tokens
print(X_train_tokens[0:50])

['000', '10', '100', '11', '113', '12', '1200', '127', '12in', '13', '130k', '14', '15', '15m', '16', '17', '1795', '18', '18k', '19', '1901', '1970s', '1974', '1995', '19m', '1m', '1st', '20', '200', '2010', '2012', '2013', '2014', '2015', '2175026', '2206960', '24', '24m', '24th', '25', '250', '26', '27', '28', '280', '30', '300', '300k', '32', '35']


In [123]:
# examine the last 50 tokens
print(X_train_tokens[-50:])

['working', 'works', 'world', 'worldwide', 'worms', 'worry', 'worse', 'worsens', 'would', 'wounded', 'wounding', 'woz', 'wozniak', 'wright', 'writer', 'writing', 'wrong', 'wrongly', 'wrote', 'wsj', 'wtc', 'wwdc', 'www', 'yang', 'year', 'years', 'yemen', 'yemeni', 'yes', 'yet', 'yeti', 'yields', 'yo', 'york', 'you', 'young', 'younger', 'youngest', 'your', 'yourselves', 'youtube', 'ypg', 'yum', 'zack', 'zehaf', 'zeppelin', 'zero', 'zeroes', 'zhejiang', 'été']


In [124]:
# Naive Bayes counts the number of times each token appears in each class
nb.feature_count_

array([[ 142.,   80.,  113., ...,   10.,    6.,    6.],
       [ 146.,   87.,  126., ...,    4.,    3.,    9.]])

In [125]:
# rows represent classes, columns represent tokens
nb.feature_count_.shape

(2, 3329)

In [126]:
ham_token_count = nb.feature_count_[0, :]
ham_token_count

array([ 142.,   80.,  113., ...,   10.,    6.,    6.])

In [127]:
# number of times each token appears across all SPAM messages
spam_token_count = nb.feature_count_[1, :]
spam_token_count

array([ 146.,   87.,  126., ...,    4.,    3.,    9.])

In [128]:
# create a DataFrame of tokens with their separate ham and spam counts
tokens = pd.DataFrame({'token':X_train_tokens, 'agree':ham_token_count, 'disagree':spam_token_count}).set_index('token')
tokens.head()

,agree,disagree
token,,
000,142.0,146.0
10,80.0,87.0
100,113.0,126.0
11,133.0,143.0
113,2.0,2.0


In [129]:
# examine 5 random DataFrame rows
tokens.sample(5, random_state=6)

,agree,disagree
token,,
mälaren,1.0,4.0
storage,9.0,9.0
taibbi,14.0,10.0
traveller,3.0,2.0
vet,5.0,10.0


In [130]:
# Naive Bayes counts the number of observations in each class
nb.class_count_

array([ 9388.,  9335.])

In [133]:
# add 1 to ham and spam counts to avoid dividing by 0
tokens['agree'] = tokens.agree + 1
tokens['disagree'] = tokens.disagree + 1
tokens.sample(5, random_state=6)

,agree,disagree
token,,
mälaren,2.0,5.0
storage,10.0,10.0
taibbi,15.0,11.0
traveller,4.0,3.0
vet,6.0,11.0


In [135]:
# convert the ham and spam counts into frequencies
tokens['agree'] = tokens.agree / nb.class_count_[0]
tokens['disagree'] = tokens.disagree / nb.class_count_[1]
tokens.sample(5, random_state=6)

,agree,disagree
token,,
mälaren,0.000213,0.000536
storage,0.001065,0.001071
taibbi,0.001598,0.001178
traveller,0.000426,0.000321
vet,0.000639,0.001178


In [136]:
# calculate the ratio of spam-to-ham for each token
tokens['spam_ratio'] = tokens.disagree / tokens.agree
tokens.sample(5, random_state=6)

,agree,disagree,spam_ratio
token,,,
mälaren,0.000213,0.000536,2.514194
storage,0.001065,0.001071,1.005678
taibbi,0.001598,0.001178,0.737497
traveller,0.000426,0.000321,0.754258
vet,0.000639,0.001178,1.843742


In [137]:
# examine the DataFrame sorted by spam_ratio
# note: use sort() instead of sort_values() for pandas 0.16.2 and earlier
tokens.sort_values('spam_ratio', ascending=False)

,agree,disagree,spam_ratio
token,,,
wildest,0.000107,0.000964,9.051098
macklemorejoinedisis,0.000107,0.000750,7.039743
reaching,0.000107,0.000643,6.034065
traded,0.000107,0.000643,6.034065
misses,0.000213,0.001178,5.531227
annual,0.000213,0.001178,5.531227
ritual,0.000213,0.001178,5.531227
retrieved,0.000107,0.000536,5.028388
forty,0.000107,0.000536,5.028388
